# Car ride-share potential in mid-size U.S. cities from geographic spread

## (notebook 3: CITY_INDEX=13)

Third notebook for the IBM Data Science Specialization on Coursera, which contains the consolidated methods and functions from the first two notebooks, to serve as a template to be used per-city.



## Installs (if needed) and imports

See notesbooks 1 and 2 for details.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math
import json

try:
    from geopy.geocoders import Nominatim
    print(Nominatim)
except:
    !conda install -c conda-forge geopy=1.18.1 --yes
    
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    import folium
    print(folium)
except:
    !conda install -c conda-forge folium=0.8.0 --yes
    
import folium
import time
    
try:
    import pandas_profiling
    print(pandas_profiling)
except:
    !conda install -c conda-forge pandas-profiling=1.4.1 --yes
    
import pandas_profiling

<class 'geopy.geocoders.osm.Nominatim'>
<module 'folium' from '/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/folium/__init__.py'>
<module 'pandas_profiling' from '/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas_profiling/__init__.py'>


## Foursquare secret

In [2]:
# (paste secret from local file)



In [3]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version

CLIENT_ID set: True
CLIENT_SECRET set: True


## Which city?

In [4]:
CITY_INDEX=13
RADIUS=800
MAX_COORDS=1000
MAP_ZOOM_LEVEL=11
MARKER_ZOOM=0.1

## All function definitions

See notebooks 1 and 2 for details.

In [5]:
print('Starting: Get list of cities')

url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })

print('---------CITIES TABLE (raw)')
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))


l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']

print('----------CITIES DATAFRAME')
print(cities_df)


def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100, verbose=True):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request (on error try four more times before giving up)
    num_tries = 0
    results = [] # assume no venues if persistent error
    
    while num_tries < 5:
        num_tries +=1
        try:
            results_raw = requests.get(url)
            results = results_raw.json()["response"]['groups'][0]['items']
        except:
            print('(err)', end='')
            time.sleep(2) # sleep for two seconds, then retry
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(results) > 0):
        nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    if verbose:
        print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    else:
        print('{}'.format(len(results)), end='. ')
    
    return(nearby_venues)


def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[], verbose=True):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            if verbose:
                print('getting coordinate {}...'.format(this_hex))
            else:
                print('({},{}):'.format(this_hex[0], this_hex[1]), end='')
                
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius=radius, limit=limit, verbose=verbose)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start,
                       city_name='(city_name)', city_state='(city_state)',
                       radius_zoom=1.0, width=600, height=600,
                       no_touch=True,
                       zoom_control=False,
                       failsafe_abort_num=4000,
                       simple_rendering=False):
    
    if zoom_control == False:
        min_zoom = zoom_start
        max_zoom = zoom_start
    else:
        min_zoom = 0
        max_zoom = 16
    
    new_map = folium.Map(
        location=[lat_orig, long_orig],
        zoom_start=zoom_start,
        width=width,
        height=height,
        control_scale=True,
        no_touch=no_touch,
        min_zoom=min_zoom,
        max_zoom=max_zoom
    )
    
    num_markers=0

    # add markers to map
    for coords, venues in venues_dict.items():
        num_markers += 1
        if (venues.shape[0] > 0) and (num_markers < failsafe_abort_num): 
            if simple_rendering:
                folium.Circle(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    color=None,
                    fill=True,
                    fill_color='#005090',
                    fill_opacity=0.7).add_to(new_map)
            else:
                label = '{}, # venues: {}'.format(coords, venues.shape[0])
                label = folium.Popup(label, parse_html=True)
                folium.CircleMarker(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    popup=label,
                    color='blue',
                    fill=True,
                    fill_color='#3186cc',
                    fill_opacity=0.7,
                    parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; left: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map


def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100, verbose=True):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    print('[test #1 of {}] (0,0):'.format(max_coords_tested), end='')
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius, verbose=verbose)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        num_coords_tested += 1
        print('[test #{} of {}]'.format(num_coords_tested, max_coords_tested), end=' ')
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit, verbose=verbose )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state


def is_indicator_venue(venue_type):
    if venue_type is None:
        return False
    
    magic_words = [
        'yoga',
        'salad',
        'coworking',
        'alternative',
        'bike',
        'fitness',
        'running',
        'jogging',
        'cycling',
        'cycle',
        'athletics',
        'gluten',
        'health',
        'recreation',
        'tennis',
        'vegetarian',
        'vegan',
        'tennis',
        'disc golf',
        'pilates',
        'share',
        'sharing',
        'incubat',
        'innovat'
    ]
    return any(substring in venue_type.lower() for substring in magic_words)

print('Perform some tests on indicator venues:')
print(is_indicator_venue(None))
print(is_indicator_venue(''))
print(is_indicator_venue('Salad Bar'))
print(is_indicator_venue('Chinese Restaurant'))
print(is_indicator_venue('Coworking space'))


def aggregate_venues_dict(venues_dict):
    venues_agg = []
    venues_types = []
    for coord, venues in venues_dict.items():
        num_venues = venues.shape[0]
        num_indicators = 0
        if num_venues > 0:
            for this_type in venues['Venue Category'].values:
                venues_types.append(this_type)
                if is_indicator_venue(this_type):
                    num_indicators += 1
        venues_agg.append([coord, num_venues, num_indicators])
    venues_agg_df = pd.DataFrame(venues_agg)
    venues_agg_df.columns = ['coord', 'num_venues', 'num_indicators']
    return venues_agg_df, set(venues_types)


def find_venues_distance_to_center(venues_agg_df):
    '''
    Appends or updates x/y and distance-to-center columns of venues_agg_df.
    Also returns some aggregates that were calculated along the way.
    '''
    
    x, y = zip(*venues_agg_df['coord'])
    total_num_venues = venues_agg_df['num_venues'].sum()
    total_num_indicators = venues_agg_df['num_indicators'].sum()
    center_x = ( x * venues_agg_df['num_venues'] ).sum() / total_num_venues
    center_y = ( y * venues_agg_df['num_venues'] ).sum() / total_num_venues
    dx = x - center_x
    dy = y - center_y
    dist2 = dx * dx + dy * dy
    venues_agg_df['dist'] = np.sqrt(dist2)
    
    return center_x, center_y, total_num_venues, total_num_indicators


def gather_distance_distribution_stats(venues_agg_df, city_index=-1, city_display='(city name, state)', return_as_list_append_string=False):
    
    center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
    num_venues_per_coord = venues_agg_df['num_venues']
    dist_per_coord = venues_agg_df['dist']
    
    # blow up the dist_per_coord series by the nunmber of venues at the coordinate
    
    dist_list = []
    for this_coord in zip(num_venues_per_coord, dist_per_coord):
        this_list = [float(this_coord[1])] * int(this_coord[0])
        dist_list.extend(this_list)
    
    dist_list_df = pd.DataFrame(dist_list)
    dist = dist_list_df[0]
    
    # now perform statistics on that new list
    
    mean = dist.mean()
    std = dist.std()
    p25 = dist.quantile(0.25)
    median = dist.quantile(0.5)
    p75 = dist.quantile(0.75)
    max_d = dist.max()
    iqr = p75 - p25
    kurt = dist.kurtosis() # "peaky-ness"
    mad = dist.mad() # mean absolute deviation
    skew = dist.skew() # skewedness
    
    if return_as_list_append_string:
        retS = '# Create list at the beginning:\n'
        retS += '#     l = []\n\n'
        retS += 'l.append(['
        retS += '{}, "{}", ({:.2f}, {:.2f}), '.format(city_index, city_display, center_x, center_y)
        retS += '{}, {}, {}, {}, '.format(total_num_venues, total_num_indicators, mean, std)
        retS += '{}, {}, {}, {}, '.format(p25, median, p75, max_d)
        retS += '{}, {}, {}, {}'.format(iqr, kurt, mad, skew)
        retS += '])\n\n'
        retS += '# Convert list to pandas DataFrame:\n'
        retS += '#     cities_stats_df = pd.DataFrame(l)\n'
        retS += '#     cities_stats_df.columns = ['
        retS += '"city index", "city name, state", "center coord", "total num venues", "total num indicators", '
        retS += '"mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", '
        retS += '"interquartile range", "kurtosis", "mean absolute deviation", "skewedness"'
        retS += ']\n'
        return retS

    else:
        return center_x, center_y, total_num_venues, total_num_indicators, \
               mean, std, p25, median, p75, max_d,                         \
               iqr, kurt, mad, skew
    

    



Starting: Get list of cities
---------CITIES TABLE (raw)
<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></s

## Quick test

In [6]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=10, verbose=False)

print("{}, {}\n".format(city_name, city_state))

venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)

print("\n{}\n".format(venues_agg_df.head(10)))

center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew \
    = gather_distance_distribution_stats(venues_agg_df)

print(center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew)

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nlist_append_string:\n\n{}'.format(list_append_string))

make_map_from_dict(
    lat_orig, long_orig, venues_dict, 13, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)


[test #1 of 10] (0,0):13. [test #2 of 10] (-1,0):1. (1,0):25. (0,-1):6. (0,1):19. (-1,1):8. (1,-1):15. [test #3 of 10] (2,0):35. (1,1):20. (2,-1):36. [test #4 of 10] (3,-1):38. (2,-2):29. (3,-2):39. [test #5 of 10] (4,-2):30. (3,-3):15. (4,-3):30. [test #6 of 10] (4,-1):29. (3,0):26. [test #7 of 10] (2,1):19. [test #8 of 10] (5,-2):9. (5,-3):22. [test #9 of 10] (4,-4):25. (5,-4):27. [test #10 of 10] (1,-2):6. (2,-3):11. Stockton, California


     coord  num_venues  num_indicators
0  (4, -3)          30               1
1  (5, -2)           9               0
2   (0, 0)          13               0
3  (-1, 0)           1               0
4   (3, 0)          26               3
5  (2, -1)          36               1
6  (2, -3)          11               0
7   (2, 1)          19               1
8  (3, -2)          39               1
9  (2, -2)          29               1

2.65853658537 -1.36210131332 533 15 1.8618913909289687 0.9726033045762776 0.9168343191287802 1.6731136755639098 2.857474939

## The real deal

In [7]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=MAX_COORDS, verbose=False)

print("\n\n{}, {}\n".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
print('Total shape: {}\n\nSample 10 venues per coordinate (unsorted):'.format(venues_agg_df.shape))
print('{}\n'.format(venues_agg_df.head(10)))

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nResulting list append string for this city:\n\n{}'.format(list_append_string))


[test #1 of 1000] (0,0):13. [test #2 of 1000] (-1,0):1. (1,0):25. (0,-1):6. (0,1):19. (-1,1):8. (1,-1):15. [test #3 of 1000] (2,0):35. (1,1):20. (2,-1):36. [test #4 of 1000] (3,-1):38. (2,-2):29. (3,-2):39. [test #5 of 1000] (4,-2):30. (3,-3):15. (4,-3):30. [test #6 of 1000] (4,-1):29. (3,0):26. [test #7 of 1000] (2,1):19. [test #8 of 1000] (5,-2):9. (5,-3):22. [test #9 of 1000] (4,-4):25. (5,-4):27. [test #10 of 1000] (1,-2):6. (2,-3):11. [test #11 of 1000] (5,-1):8. (4,0):5. [test #12 of 1000] (6,-4):27. (5,-5):25. (6,-5):40. [test #13 of 1000] (7,-5):28. (6,-6):23. (7,-6):25. [test #14 of 1000] (8,-5):16. (7,-4):24. (8,-6):26. [test #15 of 1000] (6,-3):4. [test #16 of 1000] (3,1):10. [test #17 of 1000] (9,-6):8. (8,-7):16. (9,-7):13. [test #18 of 1000] (3,-4):12. (4,-5):15. [test #19 of 1000] (5,-6):14. [test #20 of 1000] (7,-7):11. [test #21 of 1000] (8,-4):22. (7,-3):30. [test #22 of 1000] (8,-3):25. (7,-2):27. (6,-2):16. [test #23 of 1000] (8,-2):22. (7,-1):20. (6,-1):14. [test #

2. (4,3):7. [test #225 of 1000] (11,-12):5. [test #226 of 1000] [test #227 of 1000] (2,-6):6. [test #228 of 1000] [test #229 of 1000] (12,-9):7. (12,-10):4. [test #230 of 1000] (3,-7):5. (4,-7):7. [test #231 of 1000] (9,0):9. (9,-1):9. [test #232 of 1000] (10,0):4. (9,1):9. (10,-1):7. [test #233 of 1000] (10,-2):7. [test #234 of 1000] (10,1):11. (9,2):18. (8,2):8. [test #235 of 1000] (10,2):11. (9,3):15. (8,3):15. [test #236 of 1000] (10,3):11. (9,4):7. (8,4):14. [test #237 of 1000] (7,3):6. (7,4):5. [test #238 of 1000] (8,5):0. (7,5):1. [test #239 of 1000] (11,1):4. (11,0):6. [test #240 of 1000] (11,2):3. [test #241 of 1000] (11,3):2. (10,4):3. [test #242 of 1000] [test #243 of 1000] (-10,2):3. (-10,3):3. [test #244 of 1000] (11,-6):5. (11,-7):4. [test #245 of 1000] (10,-13):4. [test #246 of 1000] (11,-13):6. [test #247 of 1000] (-12,6):7. (-11,5):6. [test #248 of 1000] (9,-14):4. (10,-14):11. [test #249 of 1000] (11,-14):11. (10,-15):11. (11,-15):11. [test #250 of 1000] (12,-14):2. (

5. (-9,-1):5. [test #468 of 1000] (-11,0):4. (-10,-1):5. (-11,1):5. [test #469 of 1000] (-9,-2):2. (-8,-2):4. [test #470 of 1000] (-11,-1):0. (-10,-2):0. [test #471 of 1000] (-12,1):0. (-11,2):1. (-12,2):0. [test #472 of 1000] [test #473 of 1000] [test #474 of 1000] [test #475 of 1000] [test #476 of 1000] [test #477 of 1000] [test #478 of 1000] [test #479 of 1000] [test #480 of 1000] (13,-11):4. [test #481 of 1000] [test #482 of 1000] (12,1):0. [test #483 of 1000] [test #484 of 1000] [test #485 of 1000] [test #486 of 1000] (11,-18):2. (12,-18):3. [test #487 of 1000] [test #488 of 1000] [test #489 of 1000] (2,-8):0. [test #490 of 1000] (14,-11):6. [test #491 of 1000] (15,-11):6. (14,-12):4. (15,-12):3. [test #492 of 1000] (16,-12):3. [test #493 of 1000] [test #494 of 1000] (-14,6):0. (-14,7):0. [test #495 of 1000] (-14,8):1. [test #496 of 1000] [test #497 of 1000] (18,-9):4. [test #498 of 1000] (18,-10):4. [test #499 of 1000] (18,-11):2. [test #500 of 1000] [test #501 of 1000] [test #50

7. [test #733 of 1000] (-7,25):6. (-6,26):1. (-7,26):4. [test #734 of 1000] (-7,24):4. (-6,23):4. [test #735 of 1000] (-8,25):2. (-8,26):1. [test #736 of 1000] (-5,22):5. [test #737 of 1000] (-6,22):0. (-5,21):0. (-4,21):1. [test #738 of 1000] (-4,25):0. (-5,26):1. [test #739 of 1000] (-7,27):0. (-8,27):2. [test #740 of 1000] (-8,24):1. (-7,23):1. [test #741 of 1000] [test #742 of 1000] [test #743 of 1000] (-2,23):1. [test #744 of 1000] [test #745 of 1000] (19,-14):3. (18,-15):1. (19,-15):4. [test #746 of 1000] (20,-15):5. (19,-16):2. (20,-16):4. [test #747 of 1000] (21,-15):8. (20,-14):3. (21,-16):6. [test #748 of 1000] (22,-15):9. (21,-14):3. (22,-16):13. [test #749 of 1000] (23,-16):11. (22,-17):10. (23,-17):9. [test #750 of 1000] (24,-16):2. (23,-15):3. (24,-17):10. [test #751 of 1000] (21,-17):1. (22,-18):0. (23,-18):4. [test #752 of 1000] (25,-17):1. (24,-18):4. (25,-18):0. [test #753 of 1000] (22,-14):2. [test #754 of 1000] [test #755 of 1000] [test #756 of 1000] (20,-17):4. [te

4. (17,-22):3. (18,-22):3. [test #951 of 1000] (15,-21):2. (16,-22):4. (15,-20):3. [test #952 of 1000] (15,-22):1. (16,-23):0. (17,-23):2. [test #953 of 1000] (18,-23):4. [test #954 of 1000] (19,-23):1. (18,-24):1. (19,-24):0. [test #955 of 1000] [test #956 of 1000] (14,-20):1. (15,-19):1. (14,-19):2. [test #957 of 1000] (15,-18):1. [test #958 of 1000] (25,-15):0. (24,-14):0. [test #959 of 1000] (23,-13):0. [test #960 of 1000] (9,9):1. [test #961 of 1000] (2,20):0. (2,19):0. [test #962 of 1000] (2,21):0. (1,22):2. [test #963 of 1000] (0,23):0. [test #964 of 1000] [test #965 of 1000] (-19,44):0. (-20,44):1. [test #966 of 1000] (-21,44):1. [test #967 of 1000] (-15,29):0. (-14,28):1. [test #968 of 1000] (-4,31):2. (-4,30):1. [test #969 of 1000] (-21,39):0. [test #970 of 1000] (-22,42):1. (-22,43):0. [test #971 of 1000] (-4,32):2. [test #972 of 1000] [test #973 of 1000] (-16,42):0. [test #974 of 1000] (-9,40):1. [test #975 of 1000] (-11,27):1. (-10,27):1. [test #976 of 1000] (-12,27):2. [t

In [8]:
make_map_from_dict(
    lat_orig, long_orig, venues_dict, 11, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)
